In [1]:
import torch
from torch_geometric.data import Data
torch.cuda.is_available()

True

In [25]:
device = torch.device('cuda')

In [53]:
from torch_geometric.loader import DataLoader

# Here, it assumes that edges are directed, the size of the edge_index is 2 by num_edges
# here, it means that we have undirected edge between 0 and 1 and between 1 and 2
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
dataset = []
for i in range(10):
    x = torch.rand(3, 1)
    y = (x > 0.5).to(int)[..., 0]
    dataset.append(Data(
        x=x,
        edge_index=edge_index,
        y=y
    ).to(device))

In [56]:
dataset[2].y

tensor([0, 1, 0], device='cuda:0')

In [7]:
# check if data object fullfills requirement
data.validate(raise_on_error=True)

True

In [57]:
loader = DataLoader(dataset, batch_size=2)

In [31]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Example of a classification GCN
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [38]:
device = torch.device('cuda')
model = GCN(1, 2).to(device)

In [69]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.01, 
    weight_decay=5e-4
)
model.train()
for epoch in range(200):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = F.nll_loss(out, batch.y)
        loss.backward()
        optimizer.step()

In [77]:
model.eval()
pred = model(dataset[5]).argmax(dim=1)

In [78]:
pred

tensor([0, 0, 1], device='cuda:0')

In [76]:
dataset[5].y

tensor([0, 0, 1], device='cuda:0')